In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import FunctionTransformer
import plotly.express as px
from scipy.stats import chi2_contingency

pd.options.display.max_columns = None
#pd.options.display.max_rows = None

# Homework1 Part1

## 1.Cleaning and merging data

### a.Attendance, matches and stadium merging

#### Attendance DataFrame

In [2]:
df_attend = pd.read_csv('attendance.csv')
df_attend.head(10)

,home_team,away_team,Attendance,Date
0,Argentina,France,88966,2022-12-18
1,Croatia,Morocco,44137,2022-12-17
2,France,Morocco,68294,2022-12-14
3,Argentina,Croatia,88966,2022-12-13
4,Morocco,Portugal,44198,2022-12-10
5,England,France,68895,2022-12-10
6,Croatia,Brazil,43893,2022-12-09
7,Netherlands,Argentina,88235,2022-12-09
8,Morocco,Spain,44667,2022-12-06
9,Portugal,Switzerland,83720,2022-12-06


In [3]:
df_attend.shape

(964, 4)

In [4]:
attendance_columns = df_attend.columns
attendance_columns

Index(['home_team', 'away_team', 'Attendance', 'Date'], dtype='object')

#### Team DataFrame


  

* The team Dataframe will be a major help in the merging opeartion:

  * It will help to standarize the different spelling of the same country (eg: Türkiye & Turkey).

In [5]:
df_teams = pd.read_csv('worldcup-1.1.0/data-csv/teams.csv')
df_teams.head(5)

,key_id,team_id,team_name,team_code,federation_name,region_name,confederation_id,confederation_name,confederation_code,team_wikipedia_link,federation_wikipedia_link
0,1,T-01,Algeria,DZA,Algerian Football Federation,Africa,CF-2,Confederation of African Football,CAF,https://en.wikipedia.org/wiki/Algeria_national...,https://en.wikipedia.org/wiki/Algerian_Footbal...
1,2,T-02,Angola,AGO,Angolan Football Federation,Africa,CF-2,Confederation of African Football,CAF,https://en.wikipedia.org/wiki/Angola_national_...,https://en.wikipedia.org/wiki/Angolan_Football...
2,3,T-03,Argentina,ARG,Argentine Football Association,South America,CF-4,South American Football Confederation,CONMEBOL,https://en.wikipedia.org/wiki/Argentina_nation...,https://en.wikipedia.org/wiki/Argentine_Footba...
3,4,T-04,Australia,AUS,Football Australia,Oceania,CF-1,Asian Football Confederation,AFC,https://en.wikipedia.org/wiki/Australia_men%27...,https://en.wikipedia.org/wiki/Football_Australia
4,5,T-05,Austria,AUT,Austrian Football Association,Europe,CF-6,Union of European Football Associations,UEFA,https://en.wikipedia.org/wiki/Austria_national...,https://en.wikipedia.org/wiki/Austrian_Footbal...


* Grab from the **teams** dataframe columns[team_name, team_code], to get a dictionary of the team name and the correspondnce team 3-letter code.

In [6]:
team_name_code = dict(zip(df_teams.team_name, df_teams.team_code))

* As we've discussed there are different spelling for the same country, so we want to grab those by applying set difference to get the non-standarized team names.

In [7]:
set_difference = set(df_attend.home_team) - set(df_teams.team_name)
set_difference

{'China PR',
 "Côte d'Ivoire",
 'FR Yugoslavia',
 'Germany DR',
 'IR Iran',
 'Korea DPR',
 'Korea Republic',
 'Türkiye'}

* Now we need to build up a dictionary for the resulted country from the operation above with the correspondnce team code.

In [8]:
anamoly_countries = { "China PR" : 'CHN',
                     "Côte d'Ivoire" : 'CIV',
                     "FR Yugoslavia" : 'YUG',
                     "Germany DR" : 'DDR',
                     "IR Iran" : 'IRN',
                     "Korea DPR" : 'PRK',
                     "Korea Republic" : 'KOR',
                     "Türkiye" : 'TUR'}

In [9]:
team_name_code.update(anamoly_countries)
len(team_name_code)

93

* We need to merge based upon home_team_code and away_team_code (THANKS for you directions).
* We have to apply the map function and create them based on the dictionary which we've build.

In [10]:
df_attend['home_team_code'] = df_attend.home_team.map(team_name_code)
df_attend['away_team_code'] = df_attend.away_team.map(team_name_code)

df_attend.head(10)

,home_team,away_team,Attendance,Date,home_team_code,away_team_code
0,Argentina,France,88966,2022-12-18,ARG,FRA
1,Croatia,Morocco,44137,2022-12-17,HRV,MAR
2,France,Morocco,68294,2022-12-14,FRA,MAR
3,Argentina,Croatia,88966,2022-12-13,ARG,HRV
4,Morocco,Portugal,44198,2022-12-10,MAR,PRT
5,England,France,68895,2022-12-10,ENG,FRA
6,Croatia,Brazil,43893,2022-12-09,HRV,BRA
7,Netherlands,Argentina,88235,2022-12-09,NLD,ARG
8,Morocco,Spain,44667,2022-12-06,MAR,ESP
9,Portugal,Switzerland,83720,2022-12-06,PRT,CHE


#### Matches DataFrame

In [11]:
df_matches = pd.read_csv('worldcup-1.1.0\data-csv\matches.csv')
df_matches.head()

,key_id,tournament_id,tournament_name,match_id,match_name,stage_name,group_name,group_stage,knockout_stage,replayed,replay,match_date,match_time,stadium_id,stadium_name,city_name,country_name,home_team_id,home_team_name,home_team_code,away_team_id,away_team_name,away_team_code,score,home_team_score,away_team_score,home_team_score_margin,away_team_score_margin,extra_time,penalty_shootout,score_penalties,home_team_score_penalties,away_team_score_penalties,result,home_team_win,away_team_win,draw
0,1,WC-1930,1930 FIFA World Cup,M-1930-01,France v Mexico,group stage,Group 1,1,0,0,0,1930-07-13,15:00,S-193,Estadio Pocitos,Montevideo,Uruguay,T-28,France,FRA,T-44,Mexico,MEX,4–1,4,1,3,-3,0,0,0-0,0,0,home team win,1,0,0
1,2,WC-1930,1930 FIFA World Cup,M-1930-02,United States v Belgium,group stage,Group 4,1,0,0,0,1930-07-13,15:00,S-192,Estadio Gran Parque Central,Montevideo,Uruguay,T-80,United States,USA,T-06,Belgium,BEL,3–0,3,0,3,-3,0,0,0-0,0,0,home team win,1,0,0
2,3,WC-1930,1930 FIFA World Cup,M-1930-03,Yugoslavia v Brazil,group stage,Group 2,1,0,0,0,1930-07-14,12:45,S-192,Estadio Gran Parque Central,Montevideo,Uruguay,T-84,Yugoslavia,YUG,T-09,Brazil,BRA,2–1,2,1,1,-1,0,0,0-0,0,0,home team win,1,0,0
3,4,WC-1930,1930 FIFA World Cup,M-1930-04,Romania v Peru,group stage,Group 3,1,0,0,0,1930-07-14,14:50,S-193,Estadio Pocitos,Montevideo,Uruguay,T-59,Romania,ROU,T-54,Peru,PER,3–1,3,1,2,-2,0,0,0-0,0,0,home team win,1,0,0
4,5,WC-1930,1930 FIFA World Cup,M-1930-05,Argentina v France,group stage,Group 1,1,0,0,0,1930-07-15,16:00,S-192,Estadio Gran Parque Central,Montevideo,Uruguay,T-03,Argentina,ARG,T-28,France,FRA,1–0,1,0,1,-1,0,0,0-0,0,0,home team win,1,0,0


* Grabbing all the features of the match table, and adding to it the attendance feature of the attendance table will be useful to select the 
only columns we want based on the directions. 

In [12]:
features_of_attd_and_match = df_matches.columns.to_list()
features_of_attd_and_match.append('Attendance')
features_of_attd_and_match

['key_id',
 'tournament_id',
 'tournament_name',
 'match_id',
 'match_name',
 'stage_name',
 'group_name',
 'group_stage',
 'knockout_stage',
 'replayed',
 'replay',
 'match_date',
 'match_time',
 'stadium_id',
 'stadium_name',
 'city_name',
 'country_name',
 'home_team_id',
 'home_team_name',
 'home_team_code',
 'away_team_id',
 'away_team_name',
 'away_team_code',
 'score',
 'home_team_score',
 'away_team_score',
 'home_team_score_margin',
 'away_team_score_margin',
 'extra_time',
 'penalty_shootout',
 'score_penalties',
 'home_team_score_penalties',
 'away_team_score_penalties',
 'result',
 'home_team_win',
 'away_team_win',
 'draw',
 'Attendance']

* We are doing left join based on the features mentioned below, many rows will not get joined because the some values of home_team and away team columns are swapped for the same match.

In [13]:
merge_result = df_matches.merge(right=df_attend,
                                how='left',
                                left_on=['home_team_code', 'away_team_code', 'match_date'],
                                right_on=['home_team_code', 'away_team_code', 'Date'])


* Getting the indicies of the rows with swapped values.

In [14]:
temp = merge_result.isna().any(axis=1)
null_idx = temp[temp].index

* Perfrom a swap between home_team_code and away_team_code values for the selected indicies only.

In [15]:
hold__home_team_code = df_matches.loc[null_idx, 'home_team_code'] 

df_matches.loc[null_idx,'home_team_code'] = df_matches.loc[null_idx, 'away_team_code']
df_matches.loc[null_idx, 'away_team_code'] = hold__home_team_code


* Performing the merge operation, then selecting the columns which are mentioned in the homework.

In [16]:
merge_result_phase1 = df_matches.merge(right=df_attend,
                                how='inner',
                                left_on=['home_team_code', 'away_team_code', 'match_date'],
                                right_on=['home_team_code', 'away_team_code', 'Date'])

merge_result_phase1 = merge_result_phase1.loc[:, features_of_attd_and_match]
merge_result_phase1

,key_id,tournament_id,tournament_name,match_id,match_name,stage_name,group_name,group_stage,knockout_stage,replayed,replay,match_date,match_time,stadium_id,stadium_name,city_name,country_name,home_team_id,home_team_name,home_team_code,away_team_id,away_team_name,away_team_code,score,home_team_score,away_team_score,home_team_score_margin,away_team_score_margin,extra_time,penalty_shootout,score_penalties,home_team_score_penalties,away_team_score_penalties,result,home_team_win,away_team_win,draw,Attendance
0,1,WC-1930,1930 FIFA World Cup,M-1930-01,France v Mexico,group stage,Group 1,1,0,0,0,1930-07-13,15:00,S-193,Estadio Pocitos,Montevideo,Uruguay,T-28,France,FRA,T-44,Mexico,MEX,4–1,4,1,3,-3,0,0,0-0,0,0,home team win,1,0,0,4444
1,2,WC-1930,1930 FIFA World Cup,M-1930-02,United States v Belgium,group stage,Group 4,1,0,0,0,1930-07-13,15:00,S-192,Estadio Gran Parque Central,Montevideo,Uruguay,T-80,United States,USA,T-06,Belgium,BEL,3–0,3,0,3,-3,0,0,0-0,0,0,home team win,1,0,0,18346
2,3,WC-1930,1930 FIFA World Cup,M-1930-03,Yugoslavia v Brazil,group stage,Group 2,1,0,0,0,1930-07-14,12:45,S-192,Estadio Gran Parque Central,Montevideo,Uruguay,T-84,Yugoslavia,YUG,T-09,Brazil,BRA,2–1,2,1,1,-1,0,0,0-0,0,0,home team win,1,0,0,24059
3,4,WC-1930,1930 FIFA World Cup,M-1930-04,Romania v Peru,group stage,Group 3,1,0,0,0,1930-07-14,14:50,S-193,Estadio Pocitos,Montevideo,Uruguay,T-59,Romania,ROU,T-54,Peru,PER,3–1,3,1,2,-2,0,0,0-0,0,0,home team win,1,0,0,2549
4,5,WC-1930,1930 FIFA World Cup,M-1930-05,Argentina v France,group stage,Group 1,1,0,0,0,1930-07-15,16:00,S-192,Estadio Gran Parque Central,Montevideo,Uruguay,T-03,Argentina,ARG,T-28,France,FRA,1–0,1,0,1,-1,0,0,0-0,0,0,home team win,1,0,0,23409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
959,960,WC-2022,2022 FIFA World Cup,M-2022-60,England v France,quarter-finals,not applicable,0,1,0,0,2022-12-10,22:00,S-107,Al Bayt Stadium,Al Khor,Qatar,T-27,England,ENG,T-28,France,FRA,1–2,1,2,-1,1,0,0,0-0,0,0,away team win,0,1,0,68895
960,961,WC-2022,2022 FIFA World Cup,M-2022-61,Argentina v Croatia,semi-finals,not applicable,0,1,0,0,2022-12-13,22:00,S-114,Lusail Stadium,Lusail,Qatar,T-03,Argentina,ARG,T-17,Croatia,HRV,3–0,3,0,3,-3,0,0,0-0,0,0,home team win,1,0,0,88966
961,962,WC-2022,2022 FIFA World Cup,M-2022-62,France v Morocco,semi-finals,not applicable,0,1,0,0,2022-12-14,22:00,S-107,Al Bayt Stadium,Al Khor,Qatar,T-28,France,FRA,T-45,Morocco,MAR,2–0,2,0,2,-2,0,0,0-0,0,0,home team win,1,0,0,68294
962,963,WC-2022,2022 FIFA World Cup,M-2022-63,Croatia v Morocco,third-place match,not applicable,0,1,0,0,2022-12-17,18:00,S-110,Khalifa International Stadium,Al Rayyan,Qatar,T-17,Croatia,HRV,T-45,Morocco,MAR,2–1,2,1,1,-1,0,0,0-0,0,0,home team win,1,0,0,44137


#### Stadium DataFrame

In [17]:
df_stad = pd.read_csv('worldcup-1.1.0\data-csv\stadiums.csv')
df_stad = df_stad.loc[:, ['stadium_id', 'stadium_capacity']]

In [18]:
df_stad.shape

(193, 2)

* Merging with stadium table is an easy task, so we merge on stadium_id because it is the appropriate column to do so.

In [19]:
merge_result_phase2 = merge_result_phase1.merge(df_stad, 
                                                how='inner',
                                                on=['stadium_id']
                                                )




merge_result_phase2.head()

,key_id,tournament_id,tournament_name,match_id,match_name,stage_name,group_name,group_stage,knockout_stage,replayed,replay,match_date,match_time,stadium_id,stadium_name,city_name,country_name,home_team_id,home_team_name,home_team_code,away_team_id,away_team_name,away_team_code,score,home_team_score,away_team_score,home_team_score_margin,away_team_score_margin,extra_time,penalty_shootout,score_penalties,home_team_score_penalties,away_team_score_penalties,result,home_team_win,away_team_win,draw,Attendance,stadium_capacity
0,1,WC-1930,1930 FIFA World Cup,M-1930-01,France v Mexico,group stage,Group 1,1,0,0,0,1930-07-13,15:00,S-193,Estadio Pocitos,Montevideo,Uruguay,T-28,France,FRA,T-44,Mexico,MEX,4–1,4,1,3,-3,0,0,0-0,0,0,home team win,1,0,0,4444,10000
1,4,WC-1930,1930 FIFA World Cup,M-1930-04,Romania v Peru,group stage,Group 3,1,0,0,0,1930-07-14,14:50,S-193,Estadio Pocitos,Montevideo,Uruguay,T-59,Romania,ROU,T-54,Peru,PER,3–1,3,1,2,-2,0,0,0-0,0,0,home team win,1,0,0,2549,10000
2,2,WC-1930,1930 FIFA World Cup,M-1930-02,United States v Belgium,group stage,Group 4,1,0,0,0,1930-07-13,15:00,S-192,Estadio Gran Parque Central,Montevideo,Uruguay,T-80,United States,USA,T-06,Belgium,BEL,3–0,3,0,3,-3,0,0,0-0,0,0,home team win,1,0,0,18346,20000
3,3,WC-1930,1930 FIFA World Cup,M-1930-03,Yugoslavia v Brazil,group stage,Group 2,1,0,0,0,1930-07-14,12:45,S-192,Estadio Gran Parque Central,Montevideo,Uruguay,T-84,Yugoslavia,YUG,T-09,Brazil,BRA,2–1,2,1,1,-1,0,0,0-0,0,0,home team win,1,0,0,24059,20000
4,5,WC-1930,1930 FIFA World Cup,M-1930-05,Argentina v France,group stage,Group 1,1,0,0,0,1930-07-15,16:00,S-192,Estadio Gran Parque Central,Montevideo,Uruguay,T-03,Argentina,ARG,T-28,France,FRA,1–0,1,0,1,-1,0,0,0-0,0,0,home team win,1,0,0,23409,20000


In [20]:
features_of_attd_and_match.append('stadium_capacity')

* Final Merged table with no null values :).

In [21]:
merge_result_phase2 = merge_result_phase2.loc[:, features_of_attd_and_match]
merge_result_phase2.loc[merge_result_phase2.isna().any(axis=1), :]

,key_id,tournament_id,tournament_name,match_id,match_name,stage_name,group_name,group_stage,knockout_stage,replayed,replay,match_date,match_time,stadium_id,stadium_name,city_name,country_name,home_team_id,home_team_name,home_team_code,away_team_id,away_team_name,away_team_code,score,home_team_score,away_team_score,home_team_score_margin,away_team_score_margin,extra_time,penalty_shootout,score_penalties,home_team_score_penalties,away_team_score_penalties,result,home_team_win,away_team_win,draw,Attendance,stadium_capacity


In [22]:
final_match_df = merge_result_phase2.copy()

### b.Create players_teams table

* First we will read the tables sqauads and players tabels

In [23]:
squads_df = pd.read_csv('worldcup-1.1.0/data-csv/squads.csv')
players_df = pd.read_csv('worldcup-1.1.0/data-csv/players.csv')

 and we will check nans 

In [24]:
def checkNans(df):
    info_df = pd.DataFrame(df.dtypes)
    info_df['Nans_number'] = df.isnull().sum()
    return info_df.rename(columns={0:'Type'})

In [25]:
checkNans(squads_df)

,Type,Nans_number
key_id,int64,0
tournament_id,object,0
tournament_name,object,0
team_id,object,0
team_name,object,0
team_code,object,0
player_id,object,0
family_name,object,0
given_name,object,0
shirt_number,int64,0


In [26]:
squads_df.shape

(10973, 12)

In [27]:
checkNans(players_df)

,Type,Nans_number
key_id,int64,0
player_id,object,0
family_name,object,0
given_name,object,0
birth_date,object,1
goal_keeper,int64,0
defender,int64,0
midfielder,int64,0
forward,int64,0
count_tournaments,int64,0


In [28]:
players_df.shape

(8485, 12)

* Then we will choose the columns that we want

In [29]:
sub_squads_df = squads_df.copy()
sub_players_df = players_df.copy()
sub_players_df = players_df[['player_id','family_name','given_name','count_tournaments','list_tournaments']]
sub_squads_df = sub_squads_df[['player_id','team_name','team_code']]

and we will remove duplicate raws

In [30]:
sub_players_df = sub_players_df.drop_duplicates()
sub_squads_df = sub_squads_df.drop_duplicates()

In [31]:
print('sub_players_df')
print(f'befor drop duplicate: {players_df.shape}')
print(f'after drop duplicate: {sub_players_df.shape}')
print('sub_squads_df')
print(f'befor drop duplicate: {squads_df.shape}')
print(f'after drop duplicate: {sub_squads_df.shape}')

sub_players_df
befor drop duplicate: (8485, 12)
after drop duplicate: (8485, 5)
sub_squads_df
befor drop duplicate: (10973, 12)
after drop duplicate: (8520, 3)


here we notice that there are no duplicated raws in sub_players_df table because the player_id is unique, however in the sub_squads_df we have them because we have players that were in the same team in differents tournaments and we don't choose any of tournaments columns

* Next we will merge the tables and check nans after merge

In [32]:
merged_player_df = pd.merge(sub_players_df,sub_squads_df, on='player_id')
checkNans(merged_player_df)

,Type,Nans_number
player_id,object,0
family_name,object,0
given_name,object,0
count_tournaments,int64,0
list_tournaments,object,0
team_name,object,0
team_code,object,0


* After that we will create <strong>list_team_name and list_team_code </strong> columns

In [33]:
grouped_byID_df = merged_player_df.groupby('player_id',as_index=False).agg({'team_name':",".join,'team_code':','.join})
grouped_byID_df.shape

(8485, 3)

we grouped team_name and team_code by player_id and we will merge them with merged_player_df table after remove old team_name and team_code columns

In [34]:
final_player_df = pd.merge(merged_player_df.drop(['team_name','team_code'],axis=1),grouped_byID_df).drop_duplicates()
final_player_df = final_player_df.rename(columns={'team_name':'list_team_name','team_code':'list_team_code'})
final_player_df.shape

(8485, 7)

we notice that final_player_df and players_df has the same <strong>Columns Number (8485) </strong>

In [35]:
checkNans(final_player_df)

,Type,Nans_number
player_id,object,0
family_name,object,0
given_name,object,0
count_tournaments,int64,0
list_tournaments,object,0
list_team_name,object,0
list_team_code,object,0


* Finally we will add <strong>teams_number column</strong>

In [36]:
final_player_df['teams_number'] = final_player_df['list_team_code'].apply(lambda str: len(str.split(',')))

In [37]:
players_teams_df = final_player_df.copy()
players_teams_df.head()

,player_id,family_name,given_name,count_tournaments,list_tournaments,list_team_name,list_team_code,teams_number
0,P-08891,A'Court,Alan,1,1958,England,ENG,1
1,P-08589,Aaronson,Brenden,1,2022,United States,USA,1
2,P-04897,Abadzhiev,Stefan,1,1966,Bulgaria,BGR,1
3,P-05556,Abalo,Jean-Paul,1,2006,Togo,TGO,1
4,P-08163,Abanda,Patrice,1,1998,Cameroon,CMR,1


## 2.Feature Engineering:

### a.Create total_goals_in_match column:

* we will add match goals and penalties goal

In [38]:
final_match_df['total_goals_in_match'] = final_match_df['home_team_score']\
                                       + final_match_df['away_team_score']

In [39]:
final_match_df.head()

,key_id,tournament_id,tournament_name,match_id,match_name,stage_name,group_name,group_stage,knockout_stage,replayed,replay,match_date,match_time,stadium_id,stadium_name,city_name,country_name,home_team_id,home_team_name,home_team_code,away_team_id,away_team_name,away_team_code,score,home_team_score,away_team_score,home_team_score_margin,away_team_score_margin,extra_time,penalty_shootout,score_penalties,home_team_score_penalties,away_team_score_penalties,result,home_team_win,away_team_win,draw,Attendance,stadium_capacity,total_goals_in_match
0,1,WC-1930,1930 FIFA World Cup,M-1930-01,France v Mexico,group stage,Group 1,1,0,0,0,1930-07-13,15:00,S-193,Estadio Pocitos,Montevideo,Uruguay,T-28,France,FRA,T-44,Mexico,MEX,4–1,4,1,3,-3,0,0,0-0,0,0,home team win,1,0,0,4444,10000,5
1,4,WC-1930,1930 FIFA World Cup,M-1930-04,Romania v Peru,group stage,Group 3,1,0,0,0,1930-07-14,14:50,S-193,Estadio Pocitos,Montevideo,Uruguay,T-59,Romania,ROU,T-54,Peru,PER,3–1,3,1,2,-2,0,0,0-0,0,0,home team win,1,0,0,2549,10000,4
2,2,WC-1930,1930 FIFA World Cup,M-1930-02,United States v Belgium,group stage,Group 4,1,0,0,0,1930-07-13,15:00,S-192,Estadio Gran Parque Central,Montevideo,Uruguay,T-80,United States,USA,T-06,Belgium,BEL,3–0,3,0,3,-3,0,0,0-0,0,0,home team win,1,0,0,18346,20000,3
3,3,WC-1930,1930 FIFA World Cup,M-1930-03,Yugoslavia v Brazil,group stage,Group 2,1,0,0,0,1930-07-14,12:45,S-192,Estadio Gran Parque Central,Montevideo,Uruguay,T-84,Yugoslavia,YUG,T-09,Brazil,BRA,2–1,2,1,1,-1,0,0,0-0,0,0,home team win,1,0,0,24059,20000,3
4,5,WC-1930,1930 FIFA World Cup,M-1930-05,Argentina v France,group stage,Group 1,1,0,0,0,1930-07-15,16:00,S-192,Estadio Gran Parque Central,Montevideo,Uruguay,T-03,Argentina,ARG,T-28,France,FRA,1–0,1,0,1,-1,0,0,0-0,0,0,home team win,1,0,0,23409,20000,1


### b.Create match_for_host column:

In [40]:
final_match_df['match_for_host'] = (final_match_df['country_name'] == final_match_df['home_team_name'])\
                                 | (final_match_df['country_name'] == final_match_df['away_team_name'])
final_match_df['match_for_host'] = final_match_df['match_for_host'].astype(int)

### c.Create used_capacity_ratio column:

In [41]:
final_match_df['used_capacity_ratio'] = final_match_df['Attendance']/final_match_df['stadium_capacity']

### d.Attendance_category and elative_attendance_category Discretization:

* we will created temp_match_df to look at attendances columns summary

In [42]:
final_match_df.describe()
temp_match_df = final_match_df[['Attendance','stadium_capacity','used_capacity_ratio']]
temp_match_df.head()

,Attendance,stadium_capacity,used_capacity_ratio
0,4444,10000,0.44440
1,2549,10000,0.25490
2,18346,20000,0.91730
3,24059,20000,1.20295
4,23409,20000,1.17045


In [43]:
temp_match_df.describe()

,Attendance,stadium_capacity,used_capacity_ratio
count,964.000000,964.000000,964.000000
mean,45693.374481,55544.605809,0.831134
std,22704.131689,27396.118946,0.229490
min,2000.000000,7000.000000,0.022222
25%,31800.000000,41000.000000,0.722222
50%,42725.000000,48000.000000,0.924674
75%,60984.500000,66000.000000,0.990742
max,173850.000000,200000.000000,2.112903


* we will create attendance_category which is attendance Discretized version :

In [44]:
attendance_transformer = KBinsDiscretizer(n_bins=3,encode='ordinal',strategy='quantile')
attendance_category = attendance_transformer.fit_transform(final_match_df['Attendance'].values.reshape(-1,1))

In [45]:
np.unique(attendance_category)

array([0., 1., 2.])

In [46]:
attendance_transformer.bin_edges_[0]

array([  2000.,  36000.,  52000., 173850.])

The Ranges are:
* [2000,36000] : low
* [36000,52000] : medium
* [52000,173850] : high

* we will create relative_attendance_category which is used_capacity_ratio Discretized version :

In [47]:
used_capacity_ratio_transformer = KBinsDiscretizer(n_bins=3,encode='ordinal',strategy='quantile')
relative_attendance_category = used_capacity_ratio_transformer.fit_transform(final_match_df['used_capacity_ratio'].values.reshape(-1,1))

In [48]:
np.unique(relative_attendance_category)

array([0., 1., 2.])

In [49]:
used_capacity_ratio_transformer.bin_edges_[0]

array([0.02222222, 0.81683784, 0.97671642, 2.11290323])

The Ranges are:
* [0.02222222,0.81683784] : low
* [0.81683784,0.97671642] : medium
* [0.97671642,2.11290323] : high

* we will add attendance_category and relative_attendance_category to relative_attendance_category:

In [50]:
final_match_df['attendance_category'] = attendance_category
final_match_df['relative_attendance_category'] = relative_attendance_category
final_match_df.head()

,key_id,tournament_id,tournament_name,match_id,match_name,stage_name,group_name,group_stage,knockout_stage,replayed,replay,match_date,match_time,stadium_id,stadium_name,city_name,country_name,home_team_id,home_team_name,home_team_code,away_team_id,away_team_name,away_team_code,score,home_team_score,away_team_score,home_team_score_margin,away_team_score_margin,extra_time,penalty_shootout,score_penalties,home_team_score_penalties,away_team_score_penalties,result,home_team_win,away_team_win,draw,Attendance,stadium_capacity,total_goals_in_match,match_for_host,used_capacity_ratio,attendance_category,relative_attendance_category
0,1,WC-1930,1930 FIFA World Cup,M-1930-01,France v Mexico,group stage,Group 1,1,0,0,0,1930-07-13,15:00,S-193,Estadio Pocitos,Montevideo,Uruguay,T-28,France,FRA,T-44,Mexico,MEX,4–1,4,1,3,-3,0,0,0-0,0,0,home team win,1,0,0,4444,10000,5,0,0.44440,0.0,0.0
1,4,WC-1930,1930 FIFA World Cup,M-1930-04,Romania v Peru,group stage,Group 3,1,0,0,0,1930-07-14,14:50,S-193,Estadio Pocitos,Montevideo,Uruguay,T-59,Romania,ROU,T-54,Peru,PER,3–1,3,1,2,-2,0,0,0-0,0,0,home team win,1,0,0,2549,10000,4,0,0.25490,0.0,0.0
2,2,WC-1930,1930 FIFA World Cup,M-1930-02,United States v Belgium,group stage,Group 4,1,0,0,0,1930-07-13,15:00,S-192,Estadio Gran Parque Central,Montevideo,Uruguay,T-80,United States,USA,T-06,Belgium,BEL,3–0,3,0,3,-3,0,0,0-0,0,0,home team win,1,0,0,18346,20000,3,0,0.91730,0.0,1.0
3,3,WC-1930,1930 FIFA World Cup,M-1930-03,Yugoslavia v Brazil,group stage,Group 2,1,0,0,0,1930-07-14,12:45,S-192,Estadio Gran Parque Central,Montevideo,Uruguay,T-84,Yugoslavia,YUG,T-09,Brazil,BRA,2–1,2,1,1,-1,0,0,0-0,0,0,home team win,1,0,0,24059,20000,3,0,1.20295,0.0,2.0
4,5,WC-1930,1930 FIFA World Cup,M-1930-05,Argentina v France,group stage,Group 1,1,0,0,0,1930-07-15,16:00,S-192,Estadio Gran Parque Central,Montevideo,Uruguay,T-03,Argentina,ARG,T-28,France,FRA,1–0,1,0,1,-1,0,0,0-0,0,0,home team win,1,0,0,23409,20000,1,0,1.17045,0.0,2.0


* finally we will map values to <strong>low, medium, high</strong>:

In [51]:
categories_map = {0:'low',1:'medium',2:'high'}
final_match_df['attendance_category'] = final_match_df['attendance_category'].map(categories_map)
final_match_df['relative_attendance_category'] = final_match_df['relative_attendance_category'].map(categories_map)

In [52]:
final_match_df.head()

,key_id,tournament_id,tournament_name,match_id,match_name,stage_name,group_name,group_stage,knockout_stage,replayed,replay,match_date,match_time,stadium_id,stadium_name,city_name,country_name,home_team_id,home_team_name,home_team_code,away_team_id,away_team_name,away_team_code,score,home_team_score,away_team_score,home_team_score_margin,away_team_score_margin,extra_time,penalty_shootout,score_penalties,home_team_score_penalties,away_team_score_penalties,result,home_team_win,away_team_win,draw,Attendance,stadium_capacity,total_goals_in_match,match_for_host,used_capacity_ratio,attendance_category,relative_attendance_category
0,1,WC-1930,1930 FIFA World Cup,M-1930-01,France v Mexico,group stage,Group 1,1,0,0,0,1930-07-13,15:00,S-193,Estadio Pocitos,Montevideo,Uruguay,T-28,France,FRA,T-44,Mexico,MEX,4–1,4,1,3,-3,0,0,0-0,0,0,home team win,1,0,0,4444,10000,5,0,0.44440,low,low
1,4,WC-1930,1930 FIFA World Cup,M-1930-04,Romania v Peru,group stage,Group 3,1,0,0,0,1930-07-14,14:50,S-193,Estadio Pocitos,Montevideo,Uruguay,T-59,Romania,ROU,T-54,Peru,PER,3–1,3,1,2,-2,0,0,0-0,0,0,home team win,1,0,0,2549,10000,4,0,0.25490,low,low
2,2,WC-1930,1930 FIFA World Cup,M-1930-02,United States v Belgium,group stage,Group 4,1,0,0,0,1930-07-13,15:00,S-192,Estadio Gran Parque Central,Montevideo,Uruguay,T-80,United States,USA,T-06,Belgium,BEL,3–0,3,0,3,-3,0,0,0-0,0,0,home team win,1,0,0,18346,20000,3,0,0.91730,low,medium
3,3,WC-1930,1930 FIFA World Cup,M-1930-03,Yugoslavia v Brazil,group stage,Group 2,1,0,0,0,1930-07-14,12:45,S-192,Estadio Gran Parque Central,Montevideo,Uruguay,T-84,Yugoslavia,YUG,T-09,Brazil,BRA,2–1,2,1,1,-1,0,0,0-0,0,0,home team win,1,0,0,24059,20000,3,0,1.20295,low,high
4,5,WC-1930,1930 FIFA World Cup,M-1930-05,Argentina v France,group stage,Group 1,1,0,0,0,1930-07-15,16:00,S-192,Estadio Gran Parque Central,Montevideo,Uruguay,T-03,Argentina,ARG,T-28,France,FRA,1–0,1,0,1,-1,0,0,0-0,0,0,home team win,1,0,0,23409,20000,1,0,1.17045,low,high


### e.Create host_country_code column:

* We will use teams talbe to map between team_name and team_code(Country code)

In [53]:
teams_df = pd.read_csv('worldcup-1.1.0/data-csv/teams.csv')
teams_df = teams_df[['team_name','team_code']]
teams_df

,team_name,team_code
0,Algeria,DZA
1,Angola,AGO
2,Argentina,ARG
3,Australia,AUS
4,Austria,AUT
...,...,...
80,Uruguay,URY
81,Wales,WAL
82,West Germany,DEU
83,Yugoslavia,YUG


* We will check Nan values:

In [54]:
checkNans(teams_df)

,Type,Nans_number
team_name,object,0
team_code,object,0


* check raw duplication :

In [55]:
teams_df.drop_duplicates().dropna().shape

(85, 2)

We don't have Nans values or duplicated raws in teams table

* convert teams table into dictionary for mapping

In [56]:
teams_df_dict = teams_df.set_index('team_name').to_dict()['team_code']
teams_df_dict

{'Algeria': 'DZA',
 'Angola': 'AGO',
 'Argentina': 'ARG',
 'Australia': 'AUS',
 'Austria': 'AUT',
 'Belgium': 'BEL',
 'Bolivia': 'BOL',
 'Bosnia and Herzegovina': 'BIH',
 'Brazil': 'BRA',
 'Bulgaria': 'BGR',
 'Cameroon': 'CMR',
 'Canada': 'CAN',
 'Chile': 'CHL',
 'China': 'CHN',
 'Colombia': 'COL',
 'Costa Rica': 'CRI',
 'Croatia': 'HRV',
 'Cuba': 'CUB',
 'Czech Republic': 'CZE',
 'Czechoslovakia': 'CSK',
 'Denmark': 'DNK',
 'Dutch East Indies': 'IDN',
 'East Germany': 'DDR',
 'Ecuador': 'ECU',
 'Egypt': 'EGY',
 'El Salvador': 'SLV',
 'England': 'ENG',
 'France': 'FRA',
 'Germany': 'DEU',
 'Ghana': 'GHA',
 'Greece': 'GRC',
 'Haiti': 'HTI',
 'Honduras': 'HND',
 'Hungary': 'HUN',
 'Iceland': 'ISL',
 'Iran': 'IRN',
 'Iraq': 'IRQ',
 'Israel': 'ISR',
 'Italy': 'ITA',
 'Ivory Coast': 'CIV',
 'Jamaica': 'JAM',
 'Japan': 'JPN',
 'Kuwait': 'KWT',
 'Mexico': 'MEX',
 'Morocco': 'MAR',
 'Netherlands': 'NLD',
 'New Zealand': 'NZL',
 'Nigeria': 'NGA',
 'North Korea': 'PRK',
 'Northern Ireland': 'NIR

* Create <strong>host_country_code</strong> column:

In [57]:
final_match_df['host_country_code'] = final_match_df['country_name'].map(teams_df_dict)
final_match_df.tail()

,key_id,tournament_id,tournament_name,match_id,match_name,stage_name,group_name,group_stage,knockout_stage,replayed,replay,match_date,match_time,stadium_id,stadium_name,city_name,country_name,home_team_id,home_team_name,home_team_code,away_team_id,away_team_name,away_team_code,score,home_team_score,away_team_score,home_team_score_margin,away_team_score_margin,extra_time,penalty_shootout,score_penalties,home_team_score_penalties,away_team_score_penalties,result,home_team_win,away_team_win,draw,Attendance,stadium_capacity,total_goals_in_match,match_for_host,used_capacity_ratio,attendance_category,relative_attendance_category,host_country_code
959,921,WC-2022,2022 FIFA World Cup,M-2022-21,Tunisia v Australia,group stage,Group D,1,0,0,0,2022-11-26,13:00,S-111,Al Janoub Stadium,Al Wakrah,Qatar,T-76,Tunisia,TUN,T-04,Australia,AUS,0–1,0,1,-1,1,0,0,0-0,0,0,away team win,0,1,0,41823,44000,1,0,0.950523,medium,medium,QAT
960,929,WC-2022,2022 FIFA World Cup,M-2022-29,Cameroon v Serbia,group stage,Group G,1,0,0,0,2022-11-28,13:00,S-111,Al Janoub Stadium,Al Wakrah,Qatar,T-11,Cameroon,CMR,T-64,Serbia,SRB,3–3,3,3,0,0,0,0,0-0,0,0,draw,0,0,1,39789,44000,6,0,0.904295,medium,medium,QAT
961,937,WC-2022,2022 FIFA World Cup,M-2022-37,Australia v Denmark,group stage,Group D,1,0,0,0,2022-11-30,18:00,S-111,Al Janoub Stadium,Al Wakrah,Qatar,T-04,Australia,AUS,T-21,Denmark,DNK,1–0,1,0,1,-1,0,0,0-0,0,0,home team win,1,0,0,41232,44000,1,0,0.937091,medium,medium,QAT
962,945,WC-2022,2022 FIFA World Cup,M-2022-45,Ghana v Uruguay,group stage,Group H,1,0,0,0,2022-12-02,18:00,S-111,Al Janoub Stadium,Al Wakrah,Qatar,T-30,Ghana,GHA,T-81,Uruguay,URY,0–2,0,2,-2,2,0,0,0-0,0,0,away team win,0,1,0,43443,44000,2,0,0.987341,medium,high,QAT
963,953,WC-2022,2022 FIFA World Cup,M-2022-53,Japan v Croatia,round of 16,not applicable,0,1,0,0,2022-12-05,18:00,S-111,Al Janoub Stadium,Al Wakrah,Qatar,T-42,Japan,JPN,T-17,Croatia,HRV,1–1,1,1,0,0,1,1,1–3,1,3,away team win,0,1,0,42523,44000,2,0,0.966432,medium,medium,QAT


In [58]:
final_match_df[final_match_df.isna().any(axis=1)]

,key_id,tournament_id,tournament_name,match_id,match_name,stage_name,group_name,group_stage,knockout_stage,replayed,replay,match_date,match_time,stadium_id,stadium_name,city_name,country_name,home_team_id,home_team_name,home_team_code,away_team_id,away_team_name,away_team_code,score,home_team_score,away_team_score,home_team_score_margin,away_team_score_margin,extra_time,penalty_shootout,score_penalties,home_team_score_penalties,away_team_score_penalties,result,home_team_win,away_team_win,draw,Attendance,stadium_capacity,total_goals_in_match,match_for_host,used_capacity_ratio,attendance_category,relative_attendance_category,host_country_code


### f.Create tournament_year column:

In [59]:
final_match_df['tournament_year'] = final_match_df['tournament_name'].apply(lambda str: int(str.split()[0]))
final_match_df.head()

,key_id,tournament_id,tournament_name,match_id,match_name,stage_name,group_name,group_stage,knockout_stage,replayed,replay,match_date,match_time,stadium_id,stadium_name,city_name,country_name,home_team_id,home_team_name,home_team_code,away_team_id,away_team_name,away_team_code,score,home_team_score,away_team_score,home_team_score_margin,away_team_score_margin,extra_time,penalty_shootout,score_penalties,home_team_score_penalties,away_team_score_penalties,result,home_team_win,away_team_win,draw,Attendance,stadium_capacity,total_goals_in_match,match_for_host,used_capacity_ratio,attendance_category,relative_attendance_category,host_country_code,tournament_year
0,1,WC-1930,1930 FIFA World Cup,M-1930-01,France v Mexico,group stage,Group 1,1,0,0,0,1930-07-13,15:00,S-193,Estadio Pocitos,Montevideo,Uruguay,T-28,France,FRA,T-44,Mexico,MEX,4–1,4,1,3,-3,0,0,0-0,0,0,home team win,1,0,0,4444,10000,5,0,0.44440,low,low,URY,1930
1,4,WC-1930,1930 FIFA World Cup,M-1930-04,Romania v Peru,group stage,Group 3,1,0,0,0,1930-07-14,14:50,S-193,Estadio Pocitos,Montevideo,Uruguay,T-59,Romania,ROU,T-54,Peru,PER,3–1,3,1,2,-2,0,0,0-0,0,0,home team win,1,0,0,2549,10000,4,0,0.25490,low,low,URY,1930
2,2,WC-1930,1930 FIFA World Cup,M-1930-02,United States v Belgium,group stage,Group 4,1,0,0,0,1930-07-13,15:00,S-192,Estadio Gran Parque Central,Montevideo,Uruguay,T-80,United States,USA,T-06,Belgium,BEL,3–0,3,0,3,-3,0,0,0-0,0,0,home team win,1,0,0,18346,20000,3,0,0.91730,low,medium,URY,1930
3,3,WC-1930,1930 FIFA World Cup,M-1930-03,Yugoslavia v Brazil,group stage,Group 2,1,0,0,0,1930-07-14,12:45,S-192,Estadio Gran Parque Central,Montevideo,Uruguay,T-84,Yugoslavia,YUG,T-09,Brazil,BRA,2–1,2,1,1,-1,0,0,0-0,0,0,home team win,1,0,0,24059,20000,3,0,1.20295,low,high,URY,1930
4,5,WC-1930,1930 FIFA World Cup,M-1930-05,Argentina v France,group stage,Group 1,1,0,0,0,1930-07-15,16:00,S-192,Estadio Gran Parque Central,Montevideo,Uruguay,T-03,Argentina,ARG,T-28,France,FRA,1–0,1,0,1,-1,0,0,0-0,0,0,home team win,1,0,0,23409,20000,1,0,1.17045,low,high,URY,1930


### g.Create full_name column:

* In Fjelstul database we have some given_name column marked as <strong>'not applicable'</strong> because Some players (and managers) are commonly known by a single name (e.g., Pelé, Ronaldinho, etc.). For these players, the `family_name` variable records the name they are commonly known by and the `given_name` variable is coded `not applicable` and this was mentioned in `README.md` file in in `Data notes` part in that file

We will calculate the number of `not applicable` values in `given_name` column:

In [60]:
(final_player_df['given_name'] == 'not applicable').sum()

385

We will check if we have any of these values in `family_name` column:

In [61]:
(final_player_df['family_name'] == 'not applicable').sum()

0

* To create `full_name` column we will concatinate `given_name` with `family_name` and if we have `not applicable` values in `given_name` we don't use `given_name` value

we will create a copy of player names:

In [62]:
players_names_df = final_player_df.copy()[['given_name','family_name']]
players_names_df.head()

,given_name,family_name
0,Alan,A'Court
1,Brenden,Aaronson
2,Stefan,Abadzhiev
3,Jean-Paul,Abalo
4,Patrice,Abanda


Then we will convert `not applicable` in `given_name` to empty string: 

In [63]:
players_names_df['given_name']=players_names_df['given_name'].apply(lambda name: "" if name=='not applicable' else name)
(players_names_df['given_name'] == 'not applicable').sum()

0

In [64]:
(players_names_df['given_name'] == "").sum()

385

And we will create `full_name` column and we will add it to `final_player_df` table:

In [65]:
final_player_df['full_name'] = players_names_df['given_name'] + " " + final_player_df['family_name']

In [66]:
final_player_df[['family_name','given_name','full_name']].head()

,family_name,given_name,full_name
0,A'Court,Alan,Alan A'Court
1,Aaronson,Brenden,Brenden Aaronson
2,Abadzhiev,Stefan,Stefan Abadzhiev
3,Abalo,Jean-Paul,Jean-Paul Abalo
4,Abanda,Patrice,Patrice Abanda


### h.Create short_stage_name column:

* Create short_stage_name column:

In [67]:
final_match_df['short_stage_name'] = final_match_df['knockout_stage']
final_match_df['short_stage_name'] = final_match_df['short_stage_name'].map({1:'knockout',0:'group'})
final_match_df.head()

,key_id,tournament_id,tournament_name,match_id,match_name,stage_name,group_name,group_stage,knockout_stage,replayed,replay,match_date,match_time,stadium_id,stadium_name,city_name,country_name,home_team_id,home_team_name,home_team_code,away_team_id,away_team_name,away_team_code,score,home_team_score,away_team_score,home_team_score_margin,away_team_score_margin,extra_time,penalty_shootout,score_penalties,home_team_score_penalties,away_team_score_penalties,result,home_team_win,away_team_win,draw,Attendance,stadium_capacity,total_goals_in_match,match_for_host,used_capacity_ratio,attendance_category,relative_attendance_category,host_country_code,tournament_year,short_stage_name
0,1,WC-1930,1930 FIFA World Cup,M-1930-01,France v Mexico,group stage,Group 1,1,0,0,0,1930-07-13,15:00,S-193,Estadio Pocitos,Montevideo,Uruguay,T-28,France,FRA,T-44,Mexico,MEX,4–1,4,1,3,-3,0,0,0-0,0,0,home team win,1,0,0,4444,10000,5,0,0.44440,low,low,URY,1930,group
1,4,WC-1930,1930 FIFA World Cup,M-1930-04,Romania v Peru,group stage,Group 3,1,0,0,0,1930-07-14,14:50,S-193,Estadio Pocitos,Montevideo,Uruguay,T-59,Romania,ROU,T-54,Peru,PER,3–1,3,1,2,-2,0,0,0-0,0,0,home team win,1,0,0,2549,10000,4,0,0.25490,low,low,URY,1930,group
2,2,WC-1930,1930 FIFA World Cup,M-1930-02,United States v Belgium,group stage,Group 4,1,0,0,0,1930-07-13,15:00,S-192,Estadio Gran Parque Central,Montevideo,Uruguay,T-80,United States,USA,T-06,Belgium,BEL,3–0,3,0,3,-3,0,0,0-0,0,0,home team win,1,0,0,18346,20000,3,0,0.91730,low,medium,URY,1930,group
3,3,WC-1930,1930 FIFA World Cup,M-1930-03,Yugoslavia v Brazil,group stage,Group 2,1,0,0,0,1930-07-14,12:45,S-192,Estadio Gran Parque Central,Montevideo,Uruguay,T-84,Yugoslavia,YUG,T-09,Brazil,BRA,2–1,2,1,1,-1,0,0,0-0,0,0,home team win,1,0,0,24059,20000,3,0,1.20295,low,high,URY,1930,group
4,5,WC-1930,1930 FIFA World Cup,M-1930-05,Argentina v France,group stage,Group 1,1,0,0,0,1930-07-15,16:00,S-192,Estadio Gran Parque Central,Montevideo,Uruguay,T-03,Argentina,ARG,T-28,France,FRA,1–0,1,0,1,-1,0,0,0-0,0,0,home team win,1,0,0,23409,20000,1,0,1.17045,low,high,URY,1930,group


### i.Create winner_code:

* Read tournaments table :

In [68]:
tournaments_df = pd.read_csv('worldcup-1.1.0/data-csv/Tournaments.csv')

* Check Nans value:

In [69]:
checkNans(tournaments_df)

,Type,Nans_number
key_id,int64,0
tournament_id,object,0
tournament_name,object,0
year,int64,0
start_date,object,0
end_date,object,0
host_country,object,0
winner,object,0
host_won,int64,0
count_teams,int64,0


* Print shape and check duplicates:

In [70]:
tournaments_df.shape

(22, 18)

In [71]:
tournaments_df.drop_duplicates().shape

(22, 18)

In [72]:
tournaments_df.head()

,key_id,tournament_id,tournament_name,year,start_date,end_date,host_country,winner,host_won,count_teams,group_stage,second_group_stage,final_round,round_of_16,quarter_finals,semi_finals,third_place_match,final
0,1,WC-1930,1930 FIFA World Cup,1930,1930-07-13,1930-07-30,Uruguay,Uruguay,1,13,1,0,0,0,0,1,0,1
1,2,WC-1934,1934 FIFA World Cup,1934,1934-05-27,1934-06-10,Italy,Italy,1,16,0,0,0,1,1,1,1,1
2,3,WC-1938,1938 FIFA World Cup,1938,1938-06-04,1938-06-19,France,Italy,0,15,0,0,0,1,1,1,1,1
3,4,WC-1950,1950 FIFA World Cup,1950,1950-06-24,1950-07-16,Brazil,Uruguay,0,13,1,0,1,0,0,0,0,0
4,5,WC-1954,1954 FIFA World Cup,1954,1954-06-16,1954-07-04,Switzerland,West Germany,0,16,1,0,0,0,1,1,1,1


* we will use `teams_df_dict` from <strong>2.e</strong> to create `winner_code`:

In [73]:
tournaments_df['winner_code'] = tournaments_df['winner']
tournaments_df['winner_code'] = tournaments_df['winner_code'].map(teams_df_dict)

In [74]:
tournaments_df[['winner','winner_code']]

,winner,winner_code
0,Uruguay,URY
1,Italy,ITA
2,Italy,ITA
3,Uruguay,URY
4,West Germany,DEU
5,Brazil,BRA
6,Brazil,BRA
7,England,ENG
8,Brazil,BRA
9,West Germany,DEU


### j.Create late_goal column:

* We will read goals table:

In [75]:
goals_df = pd.read_csv('worldcup-1.1.0/data-csv/goals.csv')

* Check Nans and duplicated values:

In [76]:
goals_df.shape

(2720, 27)

In [77]:
goals_df.drop_duplicates().shape

(2720, 27)

In [78]:
checkNans(goals_df)

,Type,Nans_number
key_id,int64,0
goal_id,object,0
tournament_id,object,0
tournament_name,object,0
match_id,object,0
match_name,object,0
match_date,object,0
stage_name,object,0
group_name,object,0
team_id,object,0


In [79]:
goals_df.head()

,key_id,goal_id,tournament_id,tournament_name,match_id,match_name,match_date,stage_name,group_name,team_id,team_name,team_code,home_team,away_team,player_id,family_name,given_name,shirt_number,player_team_id,player_team_name,player_team_code,minute_label,minute_regulation,minute_stoppage,match_period,own_goal,penalty
0,1,G-0001,WC-1930,1930 FIFA World Cup,M-1930-01,France v Mexico,1930-07-13,group stage,Group 1,T-28,France,FRA,1,0,P-08962,Laurent,Lucien,0,T-28,France,FRA,19',19,0,first half,0,0
1,2,G-0002,WC-1930,1930 FIFA World Cup,M-1930-01,France v Mexico,1930-07-13,group stage,Group 1,T-28,France,FRA,1,0,P-00280,Langiller,Marcel,0,T-28,France,FRA,40',40,0,first half,0,0
2,3,G-0003,WC-1930,1930 FIFA World Cup,M-1930-01,France v Mexico,1930-07-13,group stage,Group 1,T-28,France,FRA,1,0,P-09030,Maschinot,André,0,T-28,France,FRA,43',43,0,first half,0,0
3,4,G-0004,WC-1930,1930 FIFA World Cup,M-1930-01,France v Mexico,1930-07-13,group stage,Group 1,T-44,Mexico,MEX,0,1,P-00780,Carreño,Juan,0,T-44,Mexico,MEX,70',70,0,second half,0,0
4,5,G-0005,WC-1930,1930 FIFA World Cup,M-1930-01,France v Mexico,1930-07-13,group stage,Group 1,T-28,France,FRA,1,0,P-09030,Maschinot,André,0,T-28,France,FRA,87',87,0,second half,0,0


<strong>The Late Goal :</strong> it is any goal happened in one of these times:
 *  In the minute 45 in the first half.
 *  In the stoppage time in the first half.
 *  In the minute 90 in the second half.
 *  In the stoppage time in the second half.
 *  In any time in the extra time (first extra half and second extra half).

We will reate late_goal column and we will check the minute_regulation column only because it has maximum value in the stoppage time and it can tell us the match period without check match_period column

In [80]:
goals_df['late_goal'] = (goals_df['minute_regulation'] >= 90) | (goals_df['minute_regulation'] == 45)
goals_df['late_goal'] = goals_df['late_goal'].map({True:1,False:0})

In [81]:
goals_df.head()

,key_id,goal_id,tournament_id,tournament_name,match_id,match_name,match_date,stage_name,group_name,team_id,team_name,team_code,home_team,away_team,player_id,family_name,given_name,shirt_number,player_team_id,player_team_name,player_team_code,minute_label,minute_regulation,minute_stoppage,match_period,own_goal,penalty,late_goal
0,1,G-0001,WC-1930,1930 FIFA World Cup,M-1930-01,France v Mexico,1930-07-13,group stage,Group 1,T-28,France,FRA,1,0,P-08962,Laurent,Lucien,0,T-28,France,FRA,19',19,0,first half,0,0,0
1,2,G-0002,WC-1930,1930 FIFA World Cup,M-1930-01,France v Mexico,1930-07-13,group stage,Group 1,T-28,France,FRA,1,0,P-00280,Langiller,Marcel,0,T-28,France,FRA,40',40,0,first half,0,0,0
2,3,G-0003,WC-1930,1930 FIFA World Cup,M-1930-01,France v Mexico,1930-07-13,group stage,Group 1,T-28,France,FRA,1,0,P-09030,Maschinot,André,0,T-28,France,FRA,43',43,0,first half,0,0,0
3,4,G-0004,WC-1930,1930 FIFA World Cup,M-1930-01,France v Mexico,1930-07-13,group stage,Group 1,T-44,Mexico,MEX,0,1,P-00780,Carreño,Juan,0,T-44,Mexico,MEX,70',70,0,second half,0,0,0
4,5,G-0005,WC-1930,1930 FIFA World Cup,M-1930-01,France v Mexico,1930-07-13,group stage,Group 1,T-28,France,FRA,1,0,P-09030,Maschinot,André,0,T-28,France,FRA,87',87,0,second half,0,0,0


* Calculate the numbers of the Late goals in the FIFA World Cup history:

In [82]:
print(f"the numbers of the Late goals in the FIFA World Cup history :\
 {goals_df[goals_df['late_goal'] == 1].shape[0]}")

the numbers of the Late goals in the FIFA World Cup history : 232


# Homework Part 2

## 3.Data Analysis :

In [83]:
layout_options = {
    'paper_bgcolor':"#383838",
    'plot_bgcolor':'#383838',
    'title_font': dict(color='white'),
    'legend_font': dict(color='white'),
    'yaxis':dict(color="white"),
    'xaxis':dict(color="white")
    }

### a.Attendance Analysis:

#### a.1.attendance mean and median plot:

In [84]:
# we will caloculate attendance mean and median for every tournament
att_mean_median_df = pd.DataFrame(final_match_df.groupby('tournament_year')['Attendance'].agg(['mean','median']))

In [85]:
#we will plot attendance mean and median
a_1_fig = px.line(att_mean_median_df,x=att_mean_median_df.index,y=['mean','median'],
                  width=900,height=500,title='Attendance mean and median')
a_1_fig.update_yaxes(title_text='Attendance')
a_1_fig.update_layout(**layout_options)
a_1_fig.show()


#### <strong> `The results:`</strong>

* The mean of attendance was low between 1950 and 1966 because that the World Cup was still recovering from the effects of World War II.
* The median was the lowest in 1962 because that the tournament was held in Chile, which is far from Europe and other major footballing nations. This made it difficult for fans to travel to Chile to watch the matches also the stadiums in Chile were not as big as those in Europe at that time.
* The mean and median was the highest in 1994 when the USA was the host contry.

#### <strong> `Refrences:` </strong>

* https://en.wikipedia.org/wiki/1950_FIFA_World_Cup
* https://en.wikipedia.org/wiki/1962_FIFA_World_Cup
* https://www.si.com/soccer/2018/06/01/usa-world-cup-hosting-history

#### a.2.attendance distribution Histogram plot:

* we will use <strong>`Freedman rule`</strong> which is common rule to determine the number of bins. 

In [86]:
#  this function calculate  Interquartile Range
def cal_IQR(df,column_name):
    q75, q25 = q75, q25 = df[column_name].quantile([0.75 ,0.25])
    return q75 - q25

In [87]:
#  this function calculate bins_width and bins_num using freedman rule
def cal_freedman_rule(df,column_name):
    bins_width = (2*cal_IQR(df,column_name))/(df[column_name].count()**(1/3))
    bins_num = (df[column_name].max()-df[column_name].min())/bins_width
    return int(bins_num),bins_width

In [88]:
final_match_df['Attendance'].count()

964

In [89]:
cal_freedman_rule(final_match_df,'Attendance')

(29, 5908.6723535381)

In [90]:
a_2_fig  = px.histogram(final_match_df['Attendance'],
             nbins=cal_freedman_rule(final_match_df,'Attendance')[0],
             width=1000, height=500,title='Attendance distribution')
a_2_fig  = a_2_fig.update_xaxes(title_text='Attendance')
a_2_fig.update_layout(**layout_options)
a_2_fig.show()

#### <strong> `The results:`</strong>

The largest number of attendance was between 170-180k, the smallest one was between 0-10k and in the common range of attendance was 40-50k.

#### a.2.attendance distribution box plot:

In [91]:
a_2_fig = px.box(final_match_df,
                 y='Attendance',x='tournament_year',
                 width=1000, height=500,title='Attendance distribution')
a_2_fig.update_layout(**layout_options)
a_2_fig.show()

#### <strong> `The results:`</strong>

* The largest number of attendance was in 1950 (173k) in the finall match between Uruguay and Brazil when Brazil was the host country and the main resoan for that is it was the first world cup after WW2 and also Brazil and Urguay are near to each other and that match was in Brazil and that make it easy to teams fans to come to the match also both country has larg population.
* The attendance number has the largest spread in 1950.

#### <strong> `Refrences:` </strong>

* https://en.wikipedia.org/wiki/1950_FIFA_World_Cup

### b.Goals Analysis:

In [92]:
# create datafram for goals analysis:
b_goals_df = goals_df.copy()[['family_name','given_name','tournament_name','team_name',
                            'minute_regulation','minute_stoppage','match_period','late_goal','stage_name']]

In [93]:
b_goals_df.head()

,family_name,given_name,tournament_name,team_name,minute_regulation,minute_stoppage,match_period,late_goal,stage_name
0,Laurent,Lucien,1930 FIFA World Cup,France,19,0,first half,0,group stage
1,Langiller,Marcel,1930 FIFA World Cup,France,40,0,first half,0,group stage
2,Maschinot,André,1930 FIFA World Cup,France,43,0,first half,0,group stage
3,Carreño,Juan,1930 FIFA World Cup,Mexico,70,0,second half,0,group stage
4,Maschinot,André,1930 FIFA World Cup,France,87,0,second half,0,group stage


#### b.1.Goal minute mean and median plot:

In [94]:
# Create b_goals_df dataframe for b exercises
b_goals_df['goal_minute'] = b_goals_df['minute_regulation'] + b_goals_df['minute_stoppage']

In [95]:
b_goals_df.head()

,family_name,given_name,tournament_name,team_name,minute_regulation,minute_stoppage,match_period,late_goal,stage_name,goal_minute
0,Laurent,Lucien,1930 FIFA World Cup,France,19,0,first half,0,group stage,19
1,Langiller,Marcel,1930 FIFA World Cup,France,40,0,first half,0,group stage,40
2,Maschinot,André,1930 FIFA World Cup,France,43,0,first half,0,group stage,43
3,Carreño,Juan,1930 FIFA World Cup,Mexico,70,0,second half,0,group stage,70
4,Maschinot,André,1930 FIFA World Cup,France,87,0,second half,0,group stage,87


In [96]:
mean_median_df = pd.DataFrame(b_goals_df.groupby('tournament_name')['goal_minute'].agg(['mean','median']))

In [97]:
b_1_fig = px.bar(mean_median_df,x=mean_median_df.index,y=['mean','median'],
                 width=1200,height=500,barmode='group',title='mean and median goal minute')
b_1_fig.update_yaxes(title_text='goal_minute')
b_1_fig.update_layout(**layout_options)
b_1_fig.show()

#### <strong> `The results:`</strong>

* almost all tournament has close medain an mean values.
* median and mean values in every tournament are close to each other and that mean the distribution of goal minute in every tournament is approximately symmetric.
* because  the distribution of goal minute in every tournament is approximately symmetric and all means around 50 the most goals are scored in the second half

#### b.2.Total goals in match histogram:

In [98]:
b_2_fig = px.histogram(final_match_df['total_goals_in_match'],
                       width=1000, height=500, title='Total Goals in Match')
b_2_fig = b_2_fig.update_xaxes(title_text='total_goals')
b_2_fig.update_layout(**layout_options)
b_2_fig.show()

#### <strong> `The results:`</strong>

* The most matches in world cup ended with total goals (2,3,4).
* The matches are rarely ended with number of total goals bigger than 7.
* The argest total goal was 12.

#### b.3.Calculate Most freq match period and goal minute for every tournament:

In [99]:
most_frq_df = b_goals_df.groupby('tournament_name')[['match_period','goal_minute']].apply(lambda x:x.mode())
most_frq_df.fillna('Nothing',inplace=True)
most_frq_df

match_period  goal_minute
tournament_name                                
1930 FIFA World Cup 0  second half           65
1934 FIFA World Cup 0   first half           18
                    1  second half           29
1938 FIFA World Cup 0   first half           35
                    1      Nothing           44
                    2      Nothing           89
1950 FIFA World Cup 0  second half           17
1954 FIFA World Cup 0  second half           18
                    1      Nothing           30
                    2      Nothing           54
                    3      Nothing           60
                    4      Nothing           69
                    5      Nothing           78
                    6      Nothing           84
                    7      Nothing           85
1958 FIFA World Cup 0  second half            4
                    1      Nothing           18
                    2      Nothing           24
                    3      Nothing           32
                    4      Nothing           44
                    5      Nothing           52
                    6      Nothing           55
1962 FIFA World Cup 0  second half           17
                    1      Nothing           29
                    2      Nothing           56
                    3      Nothing           73
                    4      Nothing           90
1966 FIFA World Cup 0  second half           15
                    1      Nothing           43
                    2      Nothing           75
                    3      Nothing           88
1970 FIFA World Cup 0  second half           76
1974 FIFA World Cup 0  second half           18
1978 FIFA World Cup 0   first half           43
                    1      Nothing           45
1982 FIFA World Cup 0  second half           68
                    1      Nothing           75
                    2      Nothing           83
1986 FIFA World Cup 0  second half           62
1990 FIFA World Cup 0  second half           81
1994 FIFA World Cup 0  second half           90
1998 FIFA World Cup 0  second half           46
2002 FIFA World Cup 0  second half           88
2006 FIFA World Cup 0   first half            6
                    1      Nothing           84
2010 FIFA World Cup 0  second half           89
2014 FIFA World Cup 0  second half           82
2018 FIFA World Cup 0  second half           51
2022 FIFA World Cup 0  second half           48

#### <strong> `The results:`</strong>

* The most goals are scored in the second half .
* many goals are scored at minute that is close to the beginning or the end of every half.

#### b.4.Late goals sum for every tournament:

In [100]:
late_goal_df = pd.DataFrame(b_goals_df.groupby('tournament_name')['late_goal'].agg('sum')).reset_index()
late_goal_df

,tournament_name,late_goal
0,1930 FIFA World Cup,1
1,1934 FIFA World Cup,5
2,1938 FIFA World Cup,9
3,1950 FIFA World Cup,0
4,1954 FIFA World Cup,5
5,1958 FIFA World Cup,3
6,1962 FIFA World Cup,4
7,1966 FIFA World Cup,4
8,1970 FIFA World Cup,10
9,1974 FIFA World Cup,1


In [101]:
b_4_fig = px.histogram(late_goal_df,x='tournament_name',y='late_goal',
             width=700, height=400,title='Late Goals for every tournament')
b_4_fig.update_layout(**layout_options)
b_4_fig.show()

#### b.5.Top 12 scorer in world cup :

In [102]:
# add full name column to b_goals_df
b_goals_df['given_name']=b_goals_df['given_name'].apply(lambda name: "" if name=='not applicable' else name)
b_goals_df['full_name'] = b_goals_df['given_name'] + " " + b_goals_df['family_name']

In [103]:
best_12_scorer = pd.DataFrame(b_goals_df['full_name'].value_counts().iloc[0:12]).rename(columns={'full_name':'goals'})
best_12_scorer

,goals
Miroslav Klose,16
Ronaldo,15
Gerd Müller,14
Just Fontaine,13
Lionel Messi,13
Kylian Mbappé,12
Pelé,12
Sándor Kocsis,11
Jürgen Klinsmann,11
Teófilo Cubillas,10


In [104]:
b_5_fig = px.bar(best_12_scorer,x=best_12_scorer.index,y='goals',
                 width=1200,height=500,title='Top 12 Scorer')
b_5_fig.update_xaxes(title_text='player name')
b_5_fig.update_layout(**layout_options)
b_5_fig.show()

#### <strong> `The results:`</strong>

* The best scorer scored 16 goal(Miroslav Klose).
* Kylian Mbappé	is the youngest player now in the list and he played only in the tow world cup ad scored 12 goals so that he has high probablility to be the best scorer in the world cup.

#### <strong> `Refrences:` </strong>

https://en.wikipedia.org/wiki/Kylian_Mbapp%C3%A9

#### b.6.Best scorer in every tournament:

In [105]:
# calculate the highest goals number for every scorer dataframe
temp_scorer_tournament = b_goals_df.groupby('tournament_name')['full_name'].value_counts().groupby(level=0).head(1)
temp_scorer_tournament = pd.DataFrame(temp_scorer_tournament).rename(columns={'full_name':'goals'}).reset_index()
temp_scorer_tournament = temp_scorer_tournament.drop(['full_name'],axis=1)
temp_scorer_tournament

,tournament_name,goals
0,1930 FIFA World Cup,8
1,1934 FIFA World Cup,5
2,1938 FIFA World Cup,7
3,1950 FIFA World Cup,9
4,1954 FIFA World Cup,11
5,1958 FIFA World Cup,13
6,1962 FIFA World Cup,4
7,1966 FIFA World Cup,9
8,1970 FIFA World Cup,10
9,1974 FIFA World Cup,7


In [106]:
# calculate the best scorer in every tournament dataframe and merg it with goals number
best_scorer_tournament = b_goals_df.groupby('tournament_name')['full_name'].apply(lambda x:x.mode())
best_scorer_tournament = pd.DataFrame(best_scorer_tournament)
best_scorer_tournament.groupby(level=0).head(1).reset_index().drop('level_1',axis=1)
best_scorer_tournament = pd.merge(best_scorer_tournament,temp_scorer_tournament,on='tournament_name')
best_scorer_tournament

,tournament_name,full_name,goals
0,1930 FIFA World Cup,Guillermo Stábile,8
1,1934 FIFA World Cup,Oldřich Nejedlý,5
2,1938 FIFA World Cup,Leônidas,7
3,1950 FIFA World Cup,Ademir,9
4,1954 FIFA World Cup,Sándor Kocsis,11
5,1958 FIFA World Cup,Just Fontaine,13
6,1962 FIFA World Cup,Garrincha,4
7,1962 FIFA World Cup,Vavá,4
8,1962 FIFA World Cup,Dražan Jerković,4
9,1962 FIFA World Cup,Flórián Albert,4


In [107]:
b_6_fig = px.bar(best_scorer_tournament,x='tournament_name',y='goals',color='full_name',
                 width=1200,height=500, title='best scorer in every tournament')
b_6_fig.update_layout(**layout_options)
b_6_fig.show()

#### <strong> `The results:`</strong>

* the largest number in one world cup tournament was 13 goals which are scored in 1958 world cup by Just Fontaine
* 1962 world cup has the largest number of best scorer (6 scorer)

#### b.7.Total goals in every tournament:

In [108]:
goals_tr_df = b_goals_df.groupby('tournament_name').count()

In [109]:
goals_tr_df = goals_tr_df.reset_index()[['tournament_name','family_name']].rename(columns={'family_name':'goals'})
goals_tr_df

,tournament_name,goals
0,1930 FIFA World Cup,70
1,1934 FIFA World Cup,70
2,1938 FIFA World Cup,84
3,1950 FIFA World Cup,88
4,1954 FIFA World Cup,140
5,1958 FIFA World Cup,126
6,1962 FIFA World Cup,89
7,1966 FIFA World Cup,89
8,1970 FIFA World Cup,95
9,1974 FIFA World Cup,97


In [110]:
b_7_fig = px.bar(goals_tr_df,x='tournament_name',y='goals',
              width=1200,height=500)
b_7_fig.update_layout(**layout_options)
b_7_fig.show()

#### <strong> `The results:`</strong>

* 2022 world cup has the largest number of goals (172) when 1930 and 1934 cups have the lowest one (70)
* The total number of goals is increasing in every world cup.

#### b.8.goal minute and team name strip plot :

In [111]:
strip_df = b_goals_df[['goal_minute','stage_name','team_name']]
strip_df = strip_df[strip_df['team_name'].isin(['Germany','Brazil','Italy','West Germany','East Germany'])]
strip_df.loc[strip_df['team_name'].isin(['West Germany','East Germany']),'team_name'] = 'Germany'
strip_df.shape

(602, 3)

In [112]:
b_8_fig = px.strip(strip_df,x='goal_minute',y='stage_name',facet_col='team_name',
              width=1200,height=500)
b_8_fig.show()

#### <strong> `The results:`</strong>

* Germany didn't score any goal in the final round
* Italy is the lowest team who scored in the third place match (3 goals)

## Part- 03

## C.1


* We're interested of how many times the same match occurred throught out the history. However, the values of the home_team_name and away_team_name are swapped in some rows.

* To solve this issue, in line 4 we've sorted the home_team_name and away_team_name by np.sort() function to get rid off the swapped values case.

* In line 13, there does exist a columns for the name of the match in the matches table, but we've added our own column (vs)

In [113]:
columns_of_interest = ['home_team_name', 'away_team_name']


df_matches_sorted = pd.DataFrame(np.sort(df_matches[columns_of_interest])
                                 ,columns=columns_of_interest)


df_matches_occurrence = (df_matches_sorted
                         .sort_values(by=columns_of_interest)
                         .value_counts(subset=columns_of_interest, ascending=False)
                         .reset_index(name='meeting times'))

df_matches_occurrence['vs'] = df_matches_occurrence.home_team_name + ' x ' + df_matches_occurrence.away_team_name
                 
df_matches_occurrence.loc[: 40, :]

,home_team_name,away_team_name,meeting times,vs
0,Brazil,Sweden,7,Brazil x Sweden
1,Argentina,Netherlands,6,Argentina x Netherlands
2,Brazil,Italy,5,Brazil x Italy
3,Brazil,Mexico,5,Brazil x Mexico
4,Argentina,Nigeria,5,Argentina x Nigeria
5,Argentina,England,5,Argentina x England
6,Brazil,Spain,5,Brazil x Spain
7,Argentina,Italy,5,Argentina x Italy
8,Brazil,Netherlands,5,Brazil x Netherlands
9,France,Italy,5,France x Italy


## C.2

In [114]:
px.bar(df_matches_occurrence[:10], y='meeting times', x='vs')

## D.1

* Switching teams in the world cup (not for the same tournament) is uncommon, but it did happen for many reasons we will discuss below:

  1. Puskás Ferenc for instance is orginally a Hungarian soccer player, but he played for real madird for 8 years (1958 - 1966). During this period, he
     became spanish and played for Spain in the world cup.

  2. We can see that most of the players who played for more than one country were from west west Germany. for political reasons, west and east Germany got unified under one name Germany, hence the players played for the new country after that Which is Germany.



  



In [115]:
df_player_teams = pd.read_csv('player_team.csv')
df_player_teams.loc[df_player_teams.teams_number > 1, ['given_name', 'family_name', 'list_team_name']]

,given_name,family_name,list_team_name
331,José,Altafini,"Brazil,Italy"
937,Thomas,Berthold,"West Germany,Germany"
1039,Alen,Bokšić,"Yugoslavia,Croatia"
1087,Aleksandr,Borodyuk,"Soviet Union,Russia"
1166,Andreas,Brehme,"West Germany,Germany"
1227,Guido,Buchwald,"West Germany,Germany"
2037,Attilio,Demaría,"Argentina,Italy"
3036,Sergei,Gorlukovich,"Soviet Union,Russia"
3294,Thomas,Häßler,"West Germany,Germany"
3572,Bodo,Illgner,"West Germany,Germany"


## D.2

In [116]:
df_tournaments = pd.read_csv('worldcup-1.1.0\\data-csv\\tournaments.csv')
df_tournaments.tail()

,key_id,tournament_id,tournament_name,year,start_date,end_date,host_country,winner,host_won,count_teams,group_stage,second_group_stage,final_round,round_of_16,quarter_finals,semi_finals,third_place_match,final
17,18,WC-2006,2006 FIFA World Cup,2006,2006-06-09,2006-07-09,Germany,Italy,0,32,1,0,0,1,1,1,1,1
18,19,WC-2010,2010 FIFA World Cup,2010,2010-06-11,2010-07-11,South Africa,Spain,0,32,1,0,0,1,1,1,1,1
19,20,WC-2014,2014 FIFA World Cup,2014,2014-06-12,2014-07-13,Brazil,Germany,0,32,1,0,0,1,1,1,1,1
20,21,WC-2018,2018 FIFA World Cup,2018,2018-06-14,2018-07-15,Russia,France,0,32,1,0,0,1,1,1,1,1
21,22,WC-2022,2022 FIFA World Cup,2022,2022-11-20,2022-12-18,Qatar,Argentina,0,32,1,0,0,1,1,1,1,1


In [117]:
def cramers_v(cross_tabs):
    """
    Prints the degrees of freedom, effect size thresholds, and Cramer's V value.
    
    Args:
        cross_tabs: A crosstab dataframe.
    """
    
    # effect size data frame for cramer's v function
    data = np.array([[1, .1, .3, .5],
       [2, .07, .21, .35],
       [3, .06, .17, .29],
       [4, .05,.15,.25],
       [5, .04, .13, .22]])
    sizes = pd.DataFrame(data, columns=['Degrees of Freedom', 'Small Effect', 'Medium Effect', 'Large Effect']) 
    
    # getting the chi sq. stat
    chi2 = chi2_contingency(cross_tabs)[0]
    # calculating the total number of observations
    n = cross_tabs.sum().sum()
    # getting the degrees of freedom
    dof = min(cross_tabs.shape)-1
    # calculating cramer's v
    v = np.sqrt(chi2/(n*dof))
    # printing results
    print(f'V = {v}')
    print(f'Cramer\'s V Degrees of Freedom = {dof}')
    print(f'\nEffect Size Thresholds\n{sizes}\n')

In [118]:
contingency_table = pd.crosstab(df_tournaments['winner'], df_tournaments['host_country'])
contingency_table

host_country,Argentina,Brazil,Chile,England,France,Germany,Italy,"Korea, Japan",Mexico,Qatar,Russia,South Africa,Spain,Sweden,Switzerland,United States,Uruguay,West Germany
winner,,,,,,,,,,,,,,,,,,
Argentina,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0
Brazil,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0
England,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
France,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0
Germany,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Italy,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,0,0
Spain,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
Uruguay,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
West Germany,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1


* Writing The null and alternative hypothesis:

  * H<sub>0</sub> : There is **no** relationship between the host country and the winner of the tournament.
  * H<sub>a</sub> : There is **a** relationship between the host coutnry and the winner of the tournament.

<br>

* Discussion:

   * Since p-value is 0.413 or P(Z > 138.967) = 0.413 is greater than our level of significane (alpha = 0.05), we would say that we **failed** to reject
   the null hypothesis at 95% confidience interval. in other words. we **believe** that H<sub>0</sub> is true with a confidence of 95%. 
   
  * If we look at the contingency table above we can observe that the host country does not play a factor of determining the winner in 
    the tournament. for instance, if we take Brazil's row and calculate the precentage of how many times it won, we would find that each host country where the Brazil won is equally likely for Brzail to be the champion ( prob = 1 / 5). Same case for other countries.

  * In general, for a realtionship to exist between two groups (a, b), there should be a considerable difference in frequencies for a specific value from group a the other values from group b.

In [119]:
stat, p, dof, expected = chi2_contingency(contingency_table)
print('stat=%.3f, p=%.3f, dof=%d' % (stat, p, dof))

stat=138.967, p=0.413, dof=136


* In general Even we are obtaining a high level of correlation between the two variables, that does not necessarily mean that one variable causes the other (Correlarion does not imply causation). 
Also in our case we cannot judge this type of correlation, because no country has hosted the world cup more than twice. In my opinion, If a one has hosted the world cup many time we would be able to judge this thing.

In [120]:
cramers_v(contingency_table)

V = 0.8885850175044216
Cramer's V Degrees of Freedom = 8

Effect Size Thresholds
   Degrees of Freedom  Small Effect  Medium Effect  Large Effect
0                 1.0          0.10           0.30          0.50
1                 2.0          0.07           0.21          0.35
2                 3.0          0.06           0.17          0.29
3                 4.0          0.05           0.15          0.25
4                 5.0          0.04           0.13          0.22



## D.3

* In line 4, We're selecting the rows on which the host country had a game.


In [121]:
columns_of_intrest = ['home_team_name', 'away_team_name','country_name', 'attendance_category', 'tournament_name']

df_match_attend = pd.read_csv('attendance_category.csv') 
df_match_attend_of_host = df_match_attend.loc[df_match_attend.home_team_name == df_match_attend.country_name
                                              , columns_of_intrest]


In [122]:
df_match_attend_of_host.head()

,home_team_name,away_team_name,country_name,attendance_category,tournament_name
8,Uruguay,Peru,Uruguay,high,1930 FIFA World Cup
13,Uruguay,Romania,Uruguay,high,1930 FIFA World Cup
16,Uruguay,Yugoslavia,Uruguay,high,1930 FIFA World Cup
17,Uruguay,Argentina,Uruguay,high,1930 FIFA World Cup
22,Italy,Spain,Italy,low,1934 FIFA World Cup


In [123]:
contingency_table = pd.crosstab(df_match_attend_of_host['home_team_name'], 
                                df_match_attend_of_host['attendance_category'])
contingency_table

attendance_category,high,low,medium
home_team_name,,,
Argentina,4,0,3
Brazil,10,0,1
Chile,3,1,0
England,6,0,0
France,3,2,1
Germany,6,0,0
Italy,5,3,2
Japan,2,0,1
Mexico,5,0,0


* Writing the null and the alternative hypothesis:

  * H<sub>0</sub>: There is **no** realtionship between the host match and the attendance.
  * H<sub>a</sub>: There is **a** realtionship between the host match and the attendance.
  
  <br>

* Discussion:
  * We can clearly observe that the p-value=0.01 is less than our significance level alpha=0.05. So, we reject H<sub>0</sub> and say 
    we believe that H<sub>a</sub> is true with a confidence level of 95%

  * If we observe the contingency table and calculate the row percent for each cell, we can conclude that the category of high attendancy forms the greatest precentage compared to other attendancy category on each row.

  * We can also say that the people who are from the country which is hosting the world cup form a large proportion of the attendace. Moreover, high percentage of people will not travel to another country to watch their nation's team playing, but they will attend the match if their country is the host.
   

In [124]:
stat, p, dof, expected = chi2_contingency(contingency_table)
print('stat=%.3f, p=%.3f, dof=%d' % (stat, p, dof))

stat=55.994, p=0.010, dof=34


* From cramer's value we can see indeed there is a corrleation between the two categorical variables, which supports our rejection to H<sub>0</sub>.

In [125]:
cramers_v(contingency_table)

V = 0.5672779699766338
Cramer's V Degrees of Freedom = 2

Effect Size Thresholds
   Degrees of Freedom  Small Effect  Medium Effect  Large Effect
0                 1.0          0.10           0.30          0.50
1                 2.0          0.07           0.21          0.35
2                 3.0          0.06           0.17          0.29
3                 4.0          0.05           0.15          0.25
4                 5.0          0.04           0.13          0.22



## D.4

In [126]:
contingency_table = pd.crosstab(df_match_attend['country_name'],
                                df_match_attend['attendance_category'])
contingency_table

attendance_category,high,low,medium
country_name,,,
Argentina,9,18,11
Brazil,38,13,35
Chile,10,22,0
England,10,12,10
France,17,37,28
Germany,52,11,39
Italy,22,38,9
Japan,8,8,16
Mexico,27,43,14


* Writing the null and the alternative hypothesis:

  * H<sub>0</sub>: There is **no** relationship between the attendance and the host country.
  * H<sub>a</sub>: there is **a** realtionship between the attendance and the host country.

  <br>

* Discussion:

  * it happens that for our p-value to be 0, since it is less that alpha=0.05 we reject the null hypothesis. In other words, we believe at a level of confidence of 95% that our H<sub>a</sub> is true.

  * There are many reasons for the attendance to be low compared to stadium capacity:
     1. the lack of transportation in the country. for instance, if we look at Mexico it has the highest low attendancy due to lack of transportation. In other words, this prevented the attendance to travel between stadiums on other cities within Mexico.

     2. Security In general, In my opinion Mexcio have had issues with drugs and there we a lot of gangsters at that time. So this might be a reason for the people to be concerned with, and hestiate to travel to Mexico.

     



In [127]:
stat, p, dof, expected = chi2_contingency(contingency_table)
print('stat=%.3f, p=%.3f, dof=%d' % (stat, p, dof))

stat=364.077, p=0.000, dof=34


* Also here cramer's value indicates a realtionship between the two categorical variables, which supports our rejection of H<sub>0</sub>.

In [128]:
cramers_v(contingency_table)

V = 0.43455317396048765
Cramer's V Degrees of Freedom = 2

Effect Size Thresholds
   Degrees of Freedom  Small Effect  Medium Effect  Large Effect
0                 1.0          0.10           0.30          0.50
1                 2.0          0.07           0.21          0.35
2                 3.0          0.06           0.17          0.29
3                 4.0          0.05           0.15          0.25
4                 5.0          0.04           0.13          0.22



## 4.Secret mission :

### 1.attendance ratio mean and median plot:

In [129]:
# show Attendance mean and median chart to compare with Attendance ratio chart
a_1_fig.show()

In [130]:
# we will caloculate attendance mean and median for every tournament
att_ratio_mean_median_df = pd.DataFrame(final_match_df.groupby('tournament_year')['used_capacity_ratio'].agg(['mean','median']))

In [131]:
#we will plot attendance mean and median
four_1_fig = px.line(att_ratio_mean_median_df,x=att_mean_median_df.index,y=['mean','median'],
                  width=900,height=500,title='Attendance ratio mean and median')
four_1_fig.update_yaxes(title_text='Attendance_ratio')
four_1_fig.update_layout(**layout_options)
four_1_fig.show()


#### <strong> `The results:`</strong>

* We can see that we can not rely on Attendance number to determine that if the number soccer fans is increase or not because every county has different stadiums capacity so that we will rely on Attendance ratio to remove stadiums capacity effect from chart
* We can see that both the medium and mean of Attendance ratio are increase and become closer to one and that means soccer fans number is increasing and fills more stadiums every tournament.

### 2.attendance distribution box plot:

In [132]:
four_2_fig = px.box(final_match_df,
                 y='used_capacity_ratio',x='tournament_year',
                 width=1000, height=500,title='Attendance_ratio distribution')
four_2_fig.update_layout(**layout_options)
four_2_fig.show()

#### <strong> `The results:`</strong>

* we can see that there are some matches when the number of Attendance bigger than the stadiums capacity
* 1986 cup has the busiest match in the world cup history where the number of Attendance was more than two times bigger than stadium capacity.
* In the plot above we can see that in every tournament Attendance_ratio distribution be closer and under <strong>1</strong> and that happen because that every tournament the number of Attendance is increasing and that `shift` the distribution `up` and in the other hand every country try to make stadiums bigger and that `shift` the distribution `down` so that in every tournament the distribution is `compressed` `(have less spread)` in the area that close and under <strong>1</strong> in the plot.
  
